In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pysam
from Bio.Seq import Seq
# from scipy.stats import ks_2samp
from typing import List, Tuple
import gffutils
from collections.abc import Iterable
import pybedtools

In [2]:
import pysam
# Open BAM file
# wd = "/mnt/e/Documents/APA"
wd = "/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA"
bam_files = "/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/HepG2/*.bam"
# bam_files = "/mnt/e/Documents/APA/HepG2/*.bam"
# transcriptome_file = "/mnt/e/Documents/APA/Homo_sapiens.GRCh38.cdna.all.fa"
transcriptome_file = "/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/Homo_sapiens.GRCh38.cdna.all.fa"
gtf_file = "/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/Homo_sapiens.GRCh38.109.gtf"

# Full function

In [3]:
import pysam
import pybedtools
import pandas as pd
from collections import defaultdict

def load_fasta(transcriptome_file):
    """
    Load reference FASTA file and return a Pysam FastaFile object
    """
    return pysam.FastaFile(transcriptome_file)

def get_chrom_sizes(transcriptome_file):
    """
    Get chromosome names and lengths from reference FASTA and return a dictionary
    """
    fasta = load_fasta(transcriptome_file)
    return {contig: fasta.get_reference_length(contig) for contig in fasta.references}

def group_reads(bam_file):
    """
    Group reads by transcript ID and position and return a BedTool object
    """
    bed = pybedtools.BedTool(bam_file).bam_to_bed().sort().saveas('tmp.bed')
    bed = pybedtools.BedTool('tmp.bed')
    forward = bed.filter(lambda x: x.strand == "+").saveas()
    forward = forward.groupby(g=[1,2,6], c=[3,4], o=['max','count'], full=True).saveas()
    reverse = bed.filter(lambda x: x.strand == "-").saveas()
    reverse = reverse.groupby(g=[1,3,6], c=[2,4], o=['min','count'], full=True).saveas()
    grouped_reads = forward.cat(reverse, postmerge=False).sort().saveas()
    return grouped_reads

def calculate_apa_ratios(grouped_reads):
    """
    Calculate the APA ratio for each transcript and return a dictionary
    """
    pos_dict = defaultdict(list)
    count_dict = defaultdict(list)
    strand_dict = defaultdict(list)
    for read in grouped_reads:
        transcript_id = read[0]
        pos = read.start
        count = int(read.name.split('.')[-1])
        strand = read.strand
        pos_dict[transcript_id].append(pos)
        count_dict[transcript_id].append(count)
        strand_dict[transcript_id].append(strand)
    apa_dict = {}
    for transcript_id in pos_dict:
        pos_list = pos_dict[transcript_id]
        count_list = count_dict[transcript_id]
        strand_list = strand_dict[transcript_id]
        total_count = sum(count_list)
        if total_count == 0:
            continue
        apa_pos = max(pos_list)
        apa_count = sum([count for count, pos, strand in zip(count_list, pos_list, strand_list) if pos == apa_pos and strand == '+'])
        apa_ratio = apa_count / total_count
        apa_dict[transcript_id] = {"apa_pos": apa_pos, "apa_ratio": apa_ratio, "strand": strand_list[0]}
    return apa_dict

def process_samples(bam_files, transcriptome_file):
    """
    Process multiple BAM files and return a pandas DataFrame of APA ratios
    """
    for bam_file in bam_files:
        grouped_reads = group_reads(bam_file)
        apa_dict = calculate_apa_ratios(grouped_reads)
        apa_df = pd.DataFrame.from_dict(apa_dict, orient="index")
        apa_df.index.name = "transcript_id"
        apa_df["sample"] = bam_file.split("/")[-1].split(".")[0].split("_")[0]
        apa_df["file_name"] = bam_file.split("/")[-1].split(".")[0]
        output_file = f"{wd}/{bam_file.split('/')[-1].split('.')[0]}_apa_ratios.csv"
        apa_df.to_csv(output_file, index=True)


In [88]:
import glob
bam_files = glob.glob(bam_files)
bam_files

['/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/HepG2/S3_2.bam',
 '/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/HepG2/S3_3.bam',
 '/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/HepG2/S1_1.bam',
 '/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/HepG2/S1_3.bam',
 '/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/HepG2/S3_1.bam',
 '/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/HepG2/S1_2.bam',
 '/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/HepG2/S4_1.bam',
 '/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/HepG2/S4_2.bam',


In [89]:
# process_samples(bam_files, transcriptome_file)

In [90]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sample_files = glob.glob(f"{wd}/res_HepG2/*_apa_ratios.csv")

# Load CSV files for each sample into a list of pandas DataFrames
sample_dfs = []
for sample_file in sample_files:
    sample_df = pd.read_csv(sample_file, index_col="transcript_id")
    sample_dfs.append(sample_df)

# Concatenate all sample DataFrames into a single DataFrame
df = pd.concat(sample_dfs)

In [91]:
import pandas as pd
import pybedtools
import matplotlib.pyplot as plt

# Load APA ratios data
apa_df = df

# Load GTF annotation data
gtf_df = pd.read_csv('/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/Homo_sapiens.GRCh38.109.gtf', sep="\t", comment="#", header=None,
                     names=["seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"])




/var/folders/4j/8t723zp91q3g8b3jxs_kmqgm0000gn/T/ipykernel_5583/1510415785.py:9: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gtf_df = pd.read_csv('/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/Homo_sapiens.GRCh38.109.gtf', sep="\t", comment="#", header=None,


In [3]:
import glob
bam_files = "/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/A549/*.bam"
bam_files = glob.glob(bam_files)
bam_files

['/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/A549/S3_2.bam',
 '/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/A549/S3_3.bam',
 '/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/A549/S1_1.bam',
 '/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/A549/S1_3.bam',
 '/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/A549/S3_1.bam',
 '/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/A549/S1_2.bam',
 '/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/A549/S4_1.bam',
 '/Users/seoyoonpark/Library/CloudStorage/GoogleDrive-sypark217@eunbi.co.uk/Other computers/My computer/APA/A549/S4_2.bam',
 '/Users

In [10]:
import os
import subprocess

def run_script_on_bams(bam_files):
    for bam in bam_files:
        # Convert bam file to bed format
        sample = os.path.splitext(bam)[0].split('/')[-1]
        bed_file = sample + ".bed"
        subprocess.run(f"bedtools bamtobed -i '{bam}' > '/Users/seoyoonpark/Documents/APA/bed/tmp_{bed_file}'", shell=True)
        
        # Run script on bed file
        subprocess.run(f"python /Users/seoyoonpark/Documents/APA/apa_process.py /Users/seoyoonpark/Documents/APA/bed/tmp_{bed_file} > /Users/seoyoonpark/Documents/APA/bed/{sample}.bed", shell=True)


In [11]:
run_script_on_bams(bam_files)

In [13]:
!rm /Users/seoyoonpark/Documents/APA/bed/tmp_*

In [15]:
import pandas as pd
import glob

# Define function to extract APA position and ratio information from a BED file
def extract_apa_info(bed_file):
    # Read in BED file using pandas
    df = pd.read_csv(bed_file, sep='\t', header=None, names=['transcript', 'apa_pos', 'apa_pos_end', 'apa_name', 'read_count', 'strand'])
    print(df)
    # Extract transcript ID from apa_name column
    # df['transcript_id'] = [n.split(':')[0] for n in df['apa_name']]
    # Extract APA position from apa_name column
    # df['apa_position'] = [n.split(':')[1].split('-')[0].astype(int) for n in df['apa_name']]
    # Calculate APA ratio as read count / sum of read counts for all APA sites for the transcript
    df['apa_ratio'] = df['read_count'] / df.groupby('transcript')['read_count'].transform('sum')
    # Return dataframe containing APA position and ratio information for each transcript
    return df[['transcript', 'apa_pos', 'apa_pos_end', 'apa_name', 'read_count', 'strand', 'apa_ratio']]


In [16]:
# Loop through all BED files in the directory and process each one
for bed_file in glob.glob('/Users/seoyoonpark/Documents/APA/bed/*.bed'):
    # Extract APA information for the current BED file
    apa_info = extract_apa_info(bed_file)
    # Write APA information to CSV file with the same name as the BED file, but with a .csv extension
    csv_file = bed_file.replace('.bed', '.csv')
    apa_info.to_csv(csv_file, index=False)

            transcript  apa_pos  apa_pos_end                      apa_name  \
0   ENST00000299598.11     5816         5817  ENST00000299598.11:5538-5818   
1    ENST00000419349.2     1134         1135    ENST00000419349.2:729-1135   
2    ENST00000419783.2     1143         1144    ENST00000419783.2:429-1151   
3    ENST00000420396.3      908          909     ENST00000420396.3:107-909   
4    ENST00000420396.3     1230         1231   ENST00000420396.3:1089-1231   
..                 ...      ...          ...                           ...   
69   ENST00000641557.1     1984         1985   ENST00000641557.1:1984-2332   
70   ENST00000642324.1     3082         3083   ENST00000642324.1:3073-3361   
71   ENST00000643797.1      154          155     ENST00000643797.1:154-708   
72   ENST00000645657.1     1388         1389   ENST00000645657.1:1388-1522   
73   ENST00000646881.1       13           14      ENST00000646881.1:13-589   

    read_count strand  
0           23      +  
1            5 